# LangChain サンプル 4: Chain

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** このノートブックは、SageMaker Studioの **Data Science 3.0** カーネルで動作します

### LLMChain を使用するサンプル 

In [ ]:
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrock

chat = ChatBedrock(model_id = "meta.llama3-8b-instruct-v1:0")

prompt = ChatPromptTemplate.from_messages(
    [("user", "私に {adjective} 冗談を日本語で言ってください。")],
)

chain = LLMChain(llm = chat, prompt = prompt)

response = chain.invoke({"adjective": "面白い"})
        
print(response["text"])

###  SimpleSequentialChain を使用するサンプル

In [ ]:
from langchain.chains.sequential import SimpleSequentialChain 
from langchain_core.prompts.prompt import PromptTemplate  
from langchain.chains.llm import LLMChain 
from langchain_aws import ChatBedrock   

chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0"
)


write_article_chain = LLMChain( # 記事を書くLLMChainを作成する
    llm=chat,
    prompt=PromptTemplate(
        template="{input}についての記事を書いてください。",
        input_variables=["input"],
    ),
)



translate_chain = LLMChain( # 翻訳するLLMChainを作成する
    llm=chat,
    prompt=PromptTemplate(
        template="以下の文章を英語に翻訳してください。\n{input}",
        input_variables=["input"],
    ),
)
    
sequential_chain = SimpleSequentialChain( # SimpleSequentialChainを作成する
    chains=[ # 実行するChainを指定する
        write_article_chain,
        translate_chain,
    ]
)

result = sequential_chain.invoke("飛行機の中での楽しみ方")

print(result["output"])

###  LLMRequestsChain を使用するサンプル

In [ ]:
from langchain.chains.llm_requests import LLMRequestsChain 
from langchain_core.prompts.prompt import PromptTemplate  
from langchain.chains.llm import LLMChain 
from langchain_aws import ChatBedrock   

chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" 
)

prompt = PromptTemplate( # PromptTemplateを初期化
    input_variables=["query",
                     "requests_result"],
    template="""以下の文章を元に質問に答えてください。
文章: {requests_result}
質問: {query}""",
)


llm_chain = LLMChain(
    llm=chat,
    prompt=prompt,
    verbose=True,
)

# 
chain = LLMRequestsChain(  # LLMRequestsChainを初期化
    llm_chain = llm_chain
)

print(chain.invoke({
    "query": "東京の天気について日本語で回答してください。",
    "url": "https://www.jma.go.jp/bosai/forecast/data/overview_forecast/130000.json",
}))

###  LLMChain のサンプルを LCEL の記述に変更したもの

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrock

chat = ChatBedrock(model_id = "meta.llama3-8b-instruct-v1:0")

prompt = ChatPromptTemplate.from_messages(
    [("user", "私に {adjective} 冗談を日本語で言ってください。")],
)

chain = prompt | chat | StrOutputParser()

response = chain.invoke({"adjective": "面白い"})

print(response)

### Chain 課題： SimpleSequentialChain を使用して、LLMRequestsChain で取得した結果を英語に翻訳してみよう
1. SimpleSequentialChain と LLMRequestsChain のサンプルを活用して、LLMRequestsChain で東京の天気の情報を取得した後に、その情報を英語に翻訳してみましょう。
1. SimpleSequentialChain を invoke するときのパラメータには下記の input 変数を指定して下さい。
```
input = {
    "input" :
         {
           "query": "東京の天気について日本語で回答してください。",
           "url": "https://www.jma.go.jp/bosai/forecast/data/overview_forecast/130000.json"
          }
}  
```